# Wetlands LLM + MCP Testing

Test the LLM with MCP server integration using LangChain

## Quick Start

Before running this notebook, start the local servers:

```bash
./start.sh --local
```

This starts:
- **MCP Server** (port 8001) - MotherDuck MCP server with SSE transport
- **MCP Proxy** (port 8010) - CORS-enabled proxy at `http://localhost:8010/mcp`
- **LLM Proxy** (port 8011) - OpenAI/Anthropic API proxy
- **HTTP Server** (port 8000) - Frontend

In [ ]:
import os
import json
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.tools import tool

## Configuration

In [2]:
# Load config
with open('config.json') as f:
    config = json.load(f)

# Get API key from environment
api_key = os.getenv('NRP_API_KEY')
if not api_key:
    print("WARNING: NRP_API_KEY not set!")
    api_key = input("Enter your NRP API key: ")

# MCP server URL
mcp_url =  "http://localhost:8010/mcp" #config['mcp_server_url']
llm_endpoint = config['llm_endpoint']
llm_model = config['llm_model']

print(f"LLM Endpoint: {llm_endpoint}")
print(f"Model: {llm_model}")
print(f"MCP Server: {mcp_url}")

LLM Endpoint: https://ellm.nrp-nautilus.io/v1
Model: gpt-oss
MCP Server: http://localhost:8010/mcp


## Load System Prompt

In [3]:
with open('system-prompt.md') as f:
    system_prompt = f.read()

print(f"System prompt loaded: {len(system_prompt)} characters")

System prompt loaded: 5131 characters


## Setup MCP Integration with SSE Transport

Use LangChain's official MCP adapters with SSE (Server-Sent Events) transport:

In [6]:
from langchain_mcp_adapters.client import MultiServerMCPClient

# Connect directly to the MCP server's SSE endpoint
# The motherduck MCP server exposes SSE at http://localhost:8001/sse
client = MultiServerMCPClient(
    {
        "wetlands": {
            "transport": "sse",
            "url": "http://localhost:8001/sse",
        }
    }
)

# Get tools from the MCP server
tools = await client.get_tools()

print(f"✓ Connected to MCP server!")
print(f"✓ Available tools: {[tool.name for tool in tools]}")

✓ Connected to MCP server!
✓ Available tools: ['query']


## Initialize LLM with MCP Tools

In [7]:
# Initialize LangChain LLM with MCP tools
llm = ChatOpenAI(
    base_url=llm_endpoint,
    api_key=api_key,
    model=llm_model,
    temperature=0.7
)

# Bind the MCP tools to the LLM
llm_with_tools = llm.bind_tools(tools)

print(f"LLM initialized with {len(tools)} MCP tools")

LLM initialized with 1 MCP tools


## Example: Direct Tool Test

Test an MCP tool directly (without the LLM generating the query):

In [8]:
# Find the query tool
query_tool = next((t for t in tools if t.name == "query"), None)

if query_tool:
    # Test it with a simple query
    test_query = """
    CREATE OR REPLACE SECRET s3 (TYPE S3, ENDPOINT 'minio.carlboettiger.info', URL_STYLE 'path');
    SELECT COUNT(*) as total_hexagons 
    FROM read_parquet('s3://public-wetlands/hex/**')
    WHERE Z > 0;
    """
    
    result = await query_tool.ainvoke({"query": test_query})
    print("Result:")
    print(result)
else:
    print("Query tool not found!")

Result:
+----------------+
| total_hexagons |
|     BIGINT     |
+----------------+
|   647463329    |
+----------------+


## Example: LLM with MCP Tool Calling

Now let's use the LLM to automatically generate SQL queries and call the MCP tools:

In [9]:
async def ask_wetlands_question(question: str):
    """Ask a natural language question about wetlands data"""
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=question)
    ]
    
    response = await llm_with_tools.ainvoke(messages)
    
    print(f"\n{'='*70}")
    print(f"❓ {question}")
    print(f"{'='*70}\n")
    
    # Handle tool calls
    if response.tool_calls:
        for tool_call in response.tool_calls:
            tool = next((t for t in tools if t.name == tool_call['name']), None)
            
            if tool:
                print(f"🔧 Generating and executing SQL query...\n")
                print(tool_call['args']['query'])
                print("-" * 70)
                
                # Execute the tool
                try:
                    result = await tool.ainvoke(tool_call['args'])
                    
                    print(f"\n📊 Raw result:")
                    print(result)
                    print("-" * 70)
                    
                    # Get LLM interpretation
                    messages.append(response)
                    messages.append(HumanMessage(content=f"Tool result: {result}"))
                    final = await llm.ainvoke(messages)
                    
                    print(f"\n💬 Answer:")
                    print(final.content)
                    return final.content
                except Exception as e:
                    print(f"❌ Error: {e}")
                    return None
    else:
        print(f"💬 {response.content}")
        return response.content

## Example Questions

Ask natural language questions - the LLM will generate SQL and use the MCP tools automatically:

In [10]:
# Example 1: Total wetlands area
await ask_wetlands_question("How many hectares of wetlands are there in total?")


❓ How many hectares of wetlands are there in total?

🔧 Generating and executing SQL query...

CREATE OR REPLACE SECRET s3 (
    TYPE S3,
    ENDPOINT 'minio.carlboettiger.info',
    URL_STYLE 'path'
);

SELECT 
    COUNT(*) AS total_hexagons,
    ROUND(COUNT(*) * 73.7327598, 2) AS total_hectares,
    ROUND(COUNT(*) * 0.737327598, 2) AS total_km2
FROM read_parquet('s3://public-wetlands/hex/**')
WHERE Z > 0;
----------------------------------------------------------------------

📊 Raw result:
+----------------+----------------+---------------+
| total_hexagons | total_hectares |   total_km2   |
|     BIGINT     | DECIMAL(28,2)  | DECIMAL(29,2) |
+----------------+----------------+---------------+
|   647463329    | 47739258116.47 | 477392581.16  |
+----------------+----------------+---------------+
----------------------------------------------------------------------

📊 Raw result:
+----------------+----------------+---------------+
| total_hexagons | total_hectares |   total_km2   |
|

'**Total global wetland extent (all wetland types)**  \n\n| Metric | Value |\n|--------|-------|\n| Wetland hexagons (H3\u202fresolution\u202f8) | 647,463,329 |\n| Area (hectares) | **47,739,258,116\u202fha** |\n| Area (km²) | **477,392,581\u202fkm²** |\n| Area (square miles) | **≈\u202f184,400,000\u202fmi²** |\n\n**What this means**\n\n* Each H3\u202fr8 hexagon covers about **73.73\u202fha** (0.737\u202fkm²). Multiplying the 647\u202fmillion hexagons by that constant gives the totals above.  \n* The figure represents **all wetland categories** in the global dataset (open water, palustrine, riverine, lacustrine, peatlands, coastal & other, etc.).  \n* At ~477\u202fmillion\u202fkm², the wetland area is roughly **3.2\u202ftimes the total land surface of Earth** (≈148\u202fmillion\u202fkm²). This suggests that the dataset records every wetland pixel at a very fine resolution and includes many overlapping or adjacent hexagons; it is intended as a comprehensive, spatially exhaustive invento

In [ ]:
# Example 2: Peatlands
await ask_wetlands_question("What is the total area of peatlands in square kilometers?")

In [ ]:
# Example 3: Top wetland types
await ask_wetlands_question("What are the top 5 most common wetland types by area?")

In [ ]:
# Example 4: Your custom question
await ask_wetlands_question("Compare the area of freshwater wetlands to saline wetlands")